# AI Journey 2019

Соревнование исскусственных интеллектов, автоматически решающих задачи из школьной программы.

[AI Journey 2019](https://contest.ai-journey.ru/ru/competition)

#### Постановка задачи

Необходимо разработать алгоритм, который получает на вход содержимое экзаменационного билета в электронном виде, в качестве результата предоставляет правильные ответы на вопросы из задач. В рамках соревнования, решения тестируются на вопросах из школьной программы по предмету "русский язык".

#### Формат данных

Экзаменационный билет передается решению в формате JSON. Объект экзаменационного билета содержит поле tasks со списком заданий.

Объекты задания состоят из следующих полей:


*   **id**: Идентификатор задания.
*   **text**: Текст задания. Возможно использование markdown-style форматирования. Внутри текста могут содержаться ссылки на прикрепленные файлы, например — графические иллюстрации к заданию.
*   **attachments**: Набор прикрепленных файлов (с указанием id, mime-type).
*   **meta**: Метаинформация. Произвольные пары ключ-значение. Предназначено для указания структурированных данные о вопросе. Пример: категория или подтип вопроса, источник вопроса, предмет экзамена, из которого пришел вопрос.
*   **answer**: Описание формата, в котором необходимо дать ответ. Допускаются разные типы ответов, каждый из которых имеет свои дополнительные параметры и поля:
    *   *choice*: выбор одного варианта из списка
    *   *multiple_choice*: выбор подмножества вариантов из списка
    *   *order*: расстановка вариантов в правильном порядке
    *   *matching*: верное соотнесение объектов из двух множеств
    *   *text*: ответ в виде произвольного текста
*   **score**: Максимальное количество баллов за задание.

Результат работы алгоритма записывается в виде объекта с одним полем **answers** — словарем, в котором ключи соответствуют идентификатору задания **id**, а значения — ответами на задание в соответствующем формате.









#### Пример

Пример экзаменационного билета:


```
# {
  "tasks": [
    {
      "id": "literature_3",
      "meta": {
        "language": "ru",
        "source": "ege_literature"
      },
      "text": "Сопоставьте имена и фамилии классических русских писателей.",
      "attachments": [],
      "answer": {
        "type": "matching",
        "left": [
          {"id": "A", "text": "Пушкин"},
          {"id": "B", "text": "Тургенев"},
          {"id": "C", "text": "Набоков"}
        ],
        "choices": [
          {"id": "1", "text": "Иван"},
          {"id": "2", "text": "Петр"},
          {"id": "3", "text": "Александр"},
          {"id": "4", "text": "Владимир"},
        ]
      }
    },
    ...    
  ]
}
```



Пример объекта с ответами:


```
# {
  "answers": {
    "literature_3": {"A": "3", "B": "1", "C": "4"},
    ...
  }
}
```



#### Тестовые данные

Участникам предоставляется тестовый набор экзаменационных вариантов, собранных из открытых источников.

#### Типы вопросов

Необходимо выбирать наиболее подходящий тип вопроса, таким образом, чтобы задание было наиболее машинно-читаемым.

В ряде заданий ЕГЭ используется текстовый ответ, вместо более подходящих **multiple_choice**, **matching**. Это делается из-за ограниченности бланков ЕГЭ, содержащих только лишь часть A (**choice**), часть B и С (**text**).

В некоторых заданиях используется текстовый ответ вместо choice лишь для того, чтобы задание попало в часть В и давало больше баллов, чем аналогичное задание из части A. В таком случае также необходимо использовать тип ответа choice.

In [1]:
! wget https://www.dropbox.com/s/l8z3rwu4e34u7e0/sber_and_huawei_baseline_v6.zip?dl=0

--2019-11-05 13:10:38--  https://www.dropbox.com/s/l8z3rwu4e34u7e0/sber_and_huawei_baseline_v6.zip?dl=0
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/l8z3rwu4e34u7e0/sber_and_huawei_baseline_v6.zip [переход]
--2019-11-05 13:10:38--  https://www.dropbox.com/s/raw/l8z3rwu4e34u7e0/sber_and_huawei_baseline_v6.zip
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc96f97cd2099968c124f83c7627.dl.dropboxusercontent.com/cd/0/inline/Ary_S3O78eyzjawANj2tapF8FdM7QFQZqyg4a-wdC-yvcboqc5KoVrNmx15LfSZWdDGUav6HGgMQ_xnONWAk0NjZwIupd7csq2G9JXU30EL1UG-CGS6DNOyXX3V_QmWroxg/file# [переход]
--2019-11-05 13:10:39--  https://uc96f97cd2099968c124f83c7627.dl.dropboxusercontent.com/cd/0/inline/Ary_S3O78eyzjawANj2tapF8FdM7QFQZqyg4a-wdC-yvcboqc5KoVrNmx15LfSZWdDG

In [0]:
!unzip sber_and_huawei_baseline_v6.zip?dl=0

In [4]:
% cd /content/sberbank_baseline

/content/sberbank_baseline


In [0]:
! pip install -r requirements.txt

In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Рассматриваем [Sberbank Baseline](https://github.com/sberbank-ai/ai-journey-2019/tree/master/examples/sberbank-baseline).

Используется классификатор заданий. Почти для каждого задания существует свой решатель.

Solver 1

### Указание подходящих вариантов из списка

#### **Text**
Прочитайте текст и выполните задание.


> (1) Американский психолог А. Маслоу предложил смотреть на структуру человеческих потребностей как на пирамиду, в основании <…> лежат биологические потребности, а на вершине находится потребность в самореализации. (2) Несмотря на очевидную логичность этой теории, сам А. Маслоу отмечал, что она применима к пониманию потребностей человечества в целом, но не может использоваться в отношении конкретной личности. (3) Действительно, ни одна из попыток использовать её в качестве основы для мотивации труда конкретной личности не увенчалась успехом: система ценностей каждого человека уникальна.

Укажите варианты ответов, в которых верно передана главная информация, содержащаяся в тексте.

#### Question

Выбрать подмножество:

1.   Впервые набор человеческих потребностей наиболее адекватно был описан в модели А. Маслоу, который предложил смотреть на потребности как на пирамиду.
2.   Ни одна из попыток использовать модель потребностей А. Маслоу в качестве основы для мотивации труда не увенчалась успехом в силу изначальной ошибочности и нелогичности этой модели.
3.   В основании «пирамиды потребностей» А. Маслоу лежат биологические потребности, а на её вершине находится потребность индивидуума в самореализации.
4.   Практика подтвердила, что А. Маслоу, выдвинувший идею «пирамиды потребностей», оказался прав: эта модель носит общий характер, она неприложима к конкретной личности.
5.   А. Маслоу, предложивший смотреть на структуру человеческих потребностей как на пирамиду, отмечал при этом, что она неприменима по отношению к конкретной личности, и впоследствии это подтвердилось.












#### Solution
**Указание**: Текст. Основная мысль текста.

**Решение**: В вариантах ответа 4 и 5 содержится максимально полная информация каждого из трёх предложений.

> А. Маслоу предложил смотреть на структуру человеческих потребностей как на пирамиду; эта пирамида не может использоваться в отношении конкретной личности; практика это доказала.

В варианте ответа 1 дана частная информация, которая соответствует содержанию только первого предложения; кроме того, есть дополнительная информация, которая отсутствует в тексте (впервые; наиболее адекватно).

В варианте ответа 2 содержится противоречащая тексту информация (о том, что теория А. Маслоу изначально ошибочна и нелогична).

В варианте ответа 3 содержится частная информация: она соответствует содержанию только первого предложения.

Ответ: 4, 5

#### В формате JSON

Используем тип вопроса **multiple_choice**. Поле **question.min_choices** — необязательное уточнение, говорящее о том что должен быть выбран хотя бы один из предложенных вариантов. В регламенте ЕГЭ как правило не допускается "пустого" ответа. Это ограничение (как и весь поддокумент **question**) доступно решению и может быть использовано во время тестирования.



```
{
  "id": "yandex_tutor_T7631",
  "meta": {
    "category": "ege_russian",
    "language": "ru"
  },
  "score": 1,
  "text": "Прочитайте текст и выполните задание.\n\n> (1) Американский психолог А. Маслоу предложил смотреть на структуру человеческих потребностей как на пирамиду, в основании <…> лежат биологические потребности, а на вершине находится потребность в самореализации. (2) Несмотря на очевидную логичность этой теории, сам А. Маслоу отмечал, что она применима к пониманию потребностей человечества в целом, но не может использоваться в отношении конкретной личности. (3) Действительно, ни одна из попыток использовать её в качестве основы для мотивации труда конкретной личности не увенчалась успехом: система ценностей каждого человека уникальна.\n\nУкажите варианты ответов, в которых верно передана **главная** информация, содержащаяся в тексте.",
  "attachments": [],
  "question": {
    "type": "multiple_choice",
    "choices": [
      {"id": "1", "text": "Впервые набор человеческих потребностей наиболее адекватно был описан в модели А. Маслоу, который предложил смотреть на потребности как на пирамиду."},
      {"id": "2", "text": "Ни одна из попыток использовать модель потребностей А. Маслоу в качестве основы для мотивации труда не увенчалась успехом в силу изначальной ошибочности и нелогичности этой модели."},
      {"id": "3", "text": "В основании «пирамиды потребностей» А. Маслоу лежат биологические потребности, а на её вершине находится потребность индивидуума в самореализации."},
      {"id": "4", "text": "Практика подтвердила, что А. Маслоу, выдвинувший идею «пирамиды потребностей», оказался прав: эта модель носит общий характер, она неприложима к конкретной личности."},
      {"id": "5", "text": "А. Маслоу, предложивший смотреть на структуру человеческих потребностей как на пирамиду, отмечал при этом, что она неприменима по отношению к конкретной личности, и впоследствии это подтвердилось."}
    ],
    "min_choices": 1
  },
  "solution": {
    "correct": ["4", "5"]
  }
}
```
Обратите внимание на значение поля **correct**: ["4", "5"]. В данном случае список означает единственное верное подмножество идентификаторов. В случае нескольких допустимых вариантов ответа (разных допустимых подмножеств) необходимо использовать поле **correct_variants**, например:



```
{
  "solution": {
    "correct_variants": [
      ["4", "5"],
      ["3", "4"]
    ]
  }
}
```




In [0]:
import re
import random
import operator
import pymorphy2
from nltk.tokenize import ToktokTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from solvers.utils import BertEmbedder


class Solver1(BertEmbedder):

    def __init__(self, seed=42):
        super(Solver1, self).__init__()
        self.is_train_task = False
        self.morph = pymorphy2.MorphAnalyzer()
        self.toktok = ToktokTokenizer()
        self.seed = seed
        self.init_seed()

    def init_seed(self):
        random.seed(self.seed)

    def predict(self, task):
        return self.predict_from_model(task)

    def get_num(self, text): # Сколько предложений нужно найти
        lemmas = [self.morph.parse(word)[0].normal_form for word in self.toktok.tokenize(text)]
        if 'указывать' in lemmas and 'предложение' in lemmas:
            w = lemmas[lemmas.index('указывать') + 1]  # first
            d = {'один': 1,
                 'два': 2,
                 'три': 3,
                 'четыре': 4,
                 'предложение': 1}
            if w in d:
                return d[w]
        elif 'указывать' in lemmas and 'вариант' in lemmas:
            return 'unknown'
        return 1

# Находим схожесть текста и вариантов предложений
    def compare_text_with_variants(self, text, variants, num=1):
        text_vector = self.sentence_embedding([text])
        variant_vectors = self.sentence_embedding(variants)
        i, predictions = 0, {}
        for j in variant_vectors:
            sim = cosine_similarity(text_vector[0].reshape(1, -1), j.reshape(1, -1)).flatten()[0]
            predictions[i] = sim
            i += 1
        indexes = sorted(predictions.items(), key=operator.itemgetter(1), reverse=True)[:num]
        return sorted([str(i[0] + 1) for i in indexes])

    def sent_split(self, text):
        reg = r'\(*\d+\)'
        return re.split(reg, text)

# Находим текст, варианты и сам текст задания
    def process_task(self, task):
        first_phrase, task_text = re.split(r'\(*1\)', task['text'])[:2]
        variants = [t['text'] for t in task['question']['choices']]
        text, task = "", ""
        if 'Укажите' in task_text:
            text, task = re.split('Укажите ', task_text)
            task = 'Укажите ' + task
        elif 'Укажите' in first_phrase:
            text, task = task_text, first_phrase
        return text, task, variants

    def fit(self, tasks):
        pass

    def load(self, path=""):
        pass
    
    def save(self, path=''):
        pass

    def predict_from_model(self, task, num=2):
        text, task, variants = self.process_task(task)
        result = self.compare_text_with_variants(text, variants, num=num)
        return result


Solver 2

#### В формате JSON

```
[{'attachments': [],
  'id': '2',
  'question': {'max_length': 30,
   'recommended_length': 20,
   'restriction': 'word',
   'type': 'text'},
  'score': 1,
  'solution': {'correct': 'такимобразом'},
  'text': 'Самостоятельно подберите вводное словосочетание, которое должно 
  стоять на месте пропуска в третьем (3) предложении текста.(1)В связи с тем, 
  что в прошлом женщины занимались в основном домашним хозяйством, а мужчины 
  добывали хлеб насущный себе,жене и детям, подавляющее большинство профессий 
  были мужскими: воин, пахарь, строитель, гончар, столяр, кузнец. (2)И нет 
  ничего удивительного в том, что названия почти всех профессий в языке тоже 
  мужские: рабочий, инженер, учёный, поэт, писатель, композитор, политик, 
  художник и др.  (3)<....>, женских вариантов названий этих профессий не 
  существует именно потому, что обычаи не разрешали женщинам заниматься 
  мужскими делами.'}]
```



In [0]:
import random
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize
from sklearn.exceptions import NotFittedError
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from solvers.utils import BertEmbedder
from string import punctuation
import joblib


class Solver2(BertEmbedder):

    def __init__(self, seed=42):
        super(Solver2, self).__init__()
        self.is_train_task = False
        self.seed = seed
        self.init_seed()
        self.classifier = MLPClassifier(max_iter=300)
        self.label_encoder = LabelEncoder()
        self.default_word = None
        self.fitted = False

    def init_seed(self):
        random.seed(self.seed)
        
    def save(self, path="data/models/solver2.pkl"):
        model = {"classifier": self.classifier,
                 "label_encoder": self.label_encoder,
                 "default_word": self.default_word}
        joblib.dump(model, path)

    def load(self, path="data/models/solver2.pkl"):
        model = joblib.load(path)
        self.classifier = model["classifier"]
        self.label_encoder = model["label_encoder"]
        self.default_word = model["default_word"]
        self.fitted = True

# Находим предложение, в котором пропущено слово
    @staticmethod
    def get_close_sentence(text):
        sentences = sent_tokenize(text)
        if any("<...>" in sent or "<…>" in sent for sent in sentences):
            num = next(num for num, sent in enumerate(sentences) if "<...>" in sent or "<…>" in sent)
            print(' '.join(sentences[num-1:num+1]))
            return ' '.join(sentences[num-1:num+1])
        else:
            try:
                num = next(num for num, sent in enumerate(sentences) if ("..." in sent or "…" in sent)
                           and not sent.endswith("...") and not sent.endswith("…"))
                print(' '.join(sentences[num-1:num+1]))
                return ' '.join(sentences[num - 1:num + 1])
            except StopIteration:
                return None

# Обучаем классификатор заполнять пропуски правильными словами
    def fit(self, tasks):
        X, y = list(), list()
        for task in tasks:
            text = task.get("text")
            if text is None:
                continue
            close = self.get_close_sentence(text)
            if close is None:
                continue
            correct = task["solution"]["correct_variants"] if "correct_variants" in task["solution"] else [
                task["solution"]["correct"]]
            for variant in correct:
                X.append(close)
                y.append(variant)
        self.default_word = Counter(y).most_common(1)[0][0]
        X = np.vstack(self.sentence_embedding(X))
        y = self.label_encoder.fit_transform(y)
        self.classifier.fit(X, y)
        self.fitted = True

    def predict_from_model(self, task):
        if not self.fitted:
            raise NotFittedError
        if task.get("text") is None:
            return self.default_word
        close = self.get_close_sentence(task["text"])
        if close is None:
            return self.default_word
        X = np.vstack(self.sentence_embedding([task["text"]]))
        result = self.classifier.predict(X)[0]
        result = str(list(self.label_encoder.inverse_transform([result]))[0])
        return result.strip(punctuation)


Solver 3

#### Text

Прочитайте текст и выполните задание.


> (1) Американский психолог А. Маслоу предложил смотреть на структуру человеческих потребностей как на пирамиду, в основании <…> лежат биологические потребности, а на вершине находится потребность в самореализации. (2) Несмотря на очевидную логичность этой теории, сам А. Маслоу отмечал, что она применима к пониманию потребностей человечества в целом, но не может использоваться в отношении конкретной личности. (3) Действительно, ни одна из попыток использовать её в качестве основы для мотивации труда конкретной личности не увенчалась успехом: система ценностей каждого человека уникальна.

Прочитайте фрагмент словарной статьи, в которой приводятся значения слова ОТМЕЧАТЬ. Определите значение, в котором это слово употреблено во втором (2) предложении текста.

**ОТМЕЧАТЬ**, сов.


### Question



1.   Обозначать какой-нибудь меткой. Отмечать нужное место в книге.
2.   Обращать внимание, указывать на кого-нибудь или что-нибудь, замечать. Отмечать достоинства статьи.
3.   Подчёркивать значение чего-нибудь сделанного, обычно наградив чем-нибудь. Отмечать чьи-нибудь достижения.
4.   (разг.) Отпраздновать какое-нибудь событие. Отмечать окончание института.



#### Solution

**Указание**: Лексика. Лексическое значение слова.

**Решение**: Если вместо слова «отмечать» поставить каждое из предложенных значений, единственно верным вариантом будет значение под номером 2.

Ответ: 2

#### В формате JSON

Используем тип вопроса choice, предполагающий выбор одного ответа из предложенного списка.

Текст задания и текст вариантов ответов должен быть записан в текстовом формате с Markdown форматированием. В Markdown есть специальный способ вставки цитат при помощи символов > в начале строки.

Обратите внимание: всем вариантам (choices) должен быть присвоен внутренний строковый идентификатор (id), состоящий из латинских букв и цифр. Идентификатор верного ответа записан в поле solution.correct.

Поле solution.text — необязательно. Как и весь документ solution, оно недоступно решению во время тестирования. Однако, это поле доступно в обучающей выборке и может быть полезно для обучения решений.



```
{
  "id": "yandex_tutor_T7633",
  "meta": {
    "category": "ege_russian",
    "language": "ru"
  },
  "score": 1,
  "text": "Прочитайте текст и выполните задание.\n\n> (1) Американский психолог А. Маслоу предложил смотреть на структуру человеческих потребностей как на пирамиду, в основании <…> лежат биологические потребности, а на вершине находится потребность в самореализации. (2) Несмотря на очевидную логичность этой теории, сам А. Маслоу отмечал, что она применима к пониманию потребностей человечества в целом, но не может использоваться в отношении конкретной личности. (3) Действительно, ни одна из попыток использовать её в качестве основы для мотивации труда конкретной личности не увенчалась успехом: система ценностей каждого человека уникальна.\n\nПрочитайте фрагмент словарной статьи, в которой приводятся значения слова ОТМЕЧАТЬ. Определите значение, в котором это слово употреблено во втором (2) предложении текста.\n\n**ОТМЕЧАТЬ**, сов.",
  "attachments": [],
  "question": {
    "type": "choice",
    "choices": [
      {"id": "1", "text": "Обозначать какой-нибудь меткой. Отмечать нужное место в книге."},
      {"id": "2", "text": "Обращать внимание, указывать на кого-нибудь или что-нибудь, замечать. Отмечать достоинства статьи."},
      {"id": "3", "text": "Подчёркивать значение чего-нибудь сделанного, обычно наградив чем-нибудь. Отмечать чьи-нибудь достижения."},
      {"id": "4", "text": "*(разг.)* Отпраздновать какое-нибудь событие. Отмечать окончание института."}
    ]
  },
  "solution": {
    "correct": "2",
    "text": "**Указание:** Лексика. Лексическое значение слова.\n\n**Решение:** Если вместо слова «отмечать» поставить каждое из предложенных значений, единственно верным вариантом будет значение под номером 2."
  }
}
```



#### Несколько верных ответов

Если несколько вариантов ответа допустимы как правильные, необходимо использовать поле **correct_variants**.



```
{
  "solution": {
    "correct_variants": ["2", "4"]
  }
}
```



In [0]:
import re
import operator
import random
import pymorphy2
from nltk.tokenize import ToktokTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from solvers.utils import BertEmbedder


class Solver3(BertEmbedder):

    def __init__(self, seed=42):
        super(Solver3, self).__init__()
        self.is_train_task = False
        self.morph = pymorphy2.MorphAnalyzer()
        self.toktok = ToktokTokenizer()
        self.seed = seed
        self.init_seed()

    def init_seed(self):
        random.seed(self.seed)

    def predict(self, task):
        return self.predict_from_model(task)

    def clean_text(self, text):
        newtext, logic = [], ["PREP", "CONJ", "Apro", "PRCL", "INFN", "VERB", "ADVB"]
        for token in self.toktok.tokenize(text):
            if any(tag in self.morph.parse(token)[0].tag for tag in logic):
                newtext.append(self.morph.parse(token)[0].normal_form)
        return ' '.join(newtext)

    def get_pos(self, text):
        pos, lemmas = 'word', [self.morph.parse(word)[0].normal_form for word in
                  self.toktok.tokenize(text)]
        if 'сочинительный' in lemmas:
            pos = "CCONJ"
        elif 'подчинительный' in lemmas:
            pos = "SCONJ"
        elif 'наречие' in lemmas:
            pos = "ADV"
        elif 'союзный' in lemmas:
            pos = "ADVPRO"
        elif 'местоимение' in lemmas:
            pos = "PRO"
        elif 'частица' in lemmas:
            pos = "PART"
        return pos

    def get_num(self, text):
        lemmas = [self.morph.parse(word)[0].normal_form for word in
                  self.toktok.tokenize(text)]
        if 'слово' in lemmas and 'предложение' in lemmas:
            d = {'один': 1,
                 'два': 2,
                 'три': 3,
                 'четыре': 4,
                 'первый': 1,
                 'второй': 2,
                 'третий': 3,
                 'четвертый': 4,
                 }
            for i in lemmas:
                if i in d:
                    return d[i]
        return 1

    def sent_split(self, text):
        reg = r'\(\n*\d+\n*\)'
        return re.split(reg, text)

    def compare_text_with_variants(self, word, text, variants):
        sents = self.sent_split(text)
        for sent in sents:
            lemmas = [self.morph.parse(word)[0].normal_form for word in
                  self.toktok.tokenize(text)]
            print(lemmas)
            if word.lower() in lemmas:
                text = sent
        text_vector = self.sentence_embedding([text])
        variant_vectors = self.sentence_embedding(variants)
        i, predictions = 0, {}
        for j in variant_vectors:
            sim = cosine_similarity(text_vector[0].reshape(1, -1), j.reshape(1, -1)).flatten()[0]
            predictions[i] = sim
            i += 1
        indexes = sorted(predictions.items(), key=operator.itemgetter(1), reverse=True)[:1]
        return sorted([str(i[0] + 1) for i in indexes])

# Текст, текст задания, варианты ответов, само слово
    def process_task(self, task):
        try:
            first_phrase, task_text = re.split(r'\(\n*1\n*\)', task['text'])
        except ValueError:
            first_phrase, task_text = ' '.join(re.split(r'\(\n*1\n*\)', task['text'])[:-1]), \
                                    re.split(r'\(\n*1\n*\)', task['text'])[-1]
        variants = [t['text'] for t in task['question']['choices']]
        text, task, word = "", "", ""
        if 'Определите' in task_text:
            text, task = re.split('Определите', task_text)
            task = 'Определите ' + task
            word = re.split('\.', re.split('значения слова ', text)[1])[0]
        elif 'Определите' in first_phrase:
            text, task = task_text, first_phrase
            word = re.split('\.', re.split('значения слова ', task)[1])[0]
        return text, task, variants, word

    def fit(self, tasks):
        pass

    def load(self, path="data/models/solver3.pkl"):
        pass
    
    def save(self, path='data/models/solver3.pkl'):
        pass
    
    def predict_from_model(self, task):
        text, task, variants, word = self.process_task(task)
        result = self.compare_text_with_variants(word, text, variants)
        return result


In [0]:
solver3 = Solver3()

Solver 4

#### В формате JSON

```
[{'attachments': [],
  'id': '4',
  'question': {'max_length': 30,
   'recommended_length': 20,
   'restriction': 'word',
   'type': 'text'},
  'score': 1,
  'solution': {'correct': 'отрочество'},
  'text': 'В каком слове допущена 
  ошибка в постановке ударения: 
  НЕВЕРНО выделена буква, 
  обозначающая ударный гласный звук? 
  Выпишите это слово.\nотрОчество\nмЕстностей\nоптОвый\nдиспансЕр\nсирОты'}]
```



In [0]:
import re
import os
import random
from string import punctuation


class Solver4(object):

    def __init__(self, seed=42, data_path='data/'):
        self.is_train_task = False
        self.seed = seed
        self.init_seed()
        # 'agi_stress.txt' - хранит слова с правильно расставленными ударениями
        self.stress = open(os.path.join(data_path, 'agi_stress.txt'), 'r', encoding='utf8').read().split('\n')[:-1]

    def init_seed(self):
        random.seed(self.seed)

    def predict(self, task):
        return self.predict_from_model(task)

    def compare_text_with_variants(self, variants, task_type='incorrect'):
        result = ''
        if task_type == 'incorrect':
            for variant in variants:
                if variant not in self.stress:
                    result = variant
        else:
            for variant in variants:
                if variant in self.stress:
                    result = variant
        if not variants:
            return ''
        if not result:
            result = random.choice(variants)
        return result.lower().strip(punctuation)

    def process_task(self, task):
        task_text = re.split(r'\n', task['text'])
        variants = task_text[1:-1]
        if 'Выпишите' in task_text[-1]:
            task = task_text[0] + task_text[-1]
        else:
            task = task_text[0]
        if 'неверно' in task.lower():
            task_type = 'incorrect'
        else:
            task_type = 'correct'
        return task_type, task, variants

    def fit(self, tasks):
        pass

    def load(self, path="data/models/solver4.pkl"):
        pass

    def save(self, path="data/models/solver4.pkl"):
        pass

    def predict_from_model(self, task):
        task_type, task, variants = self.process_task(task)
        result = self.compare_text_with_variants(variants, task_type)
        return result.strip()


Solver 5

#### Ответ в форме слова

#### Text
В одном из приведённых ниже предложений неверно употреблено выделенное слово. Исправьте лексическую ошибку, подобрав к выделенному слову пароним.



*   Работа молодого актёра небезупречна, однако в силу его склонности к созданию ЦЕЛОСТНОГО образа ему удалось подчеркнуть способность героя естественно откликаться на просьбы окружающих.
*   Пьеса довольно СЦЕНИЧНА и прекрасно сыграна.
*   Маятник ОТКЛОНИЛСЯ примерно на шестьдесят градусов.
*   Наше основное РАЗЛИЧИЕ друг от друга состоит не во внешности, а в восприятии мира как такового.
*   На ближайший месяц утверждён новый ПРОИЗВОДСТВЕННЫЙ план.
Запишите подобранное слово.

Запишите подобранное слово.

**Question** (text)
Необходимо записать одно слово.

**Solution**
Указание: Лексические нормы.

**Решение**: Вместо слова различие в данном контексте необходимо употребить слово отличие, поскольку далее используется предлог от (отличие чего-либо от чего либо).

Ответ: отличие

#### В формате JSON
Используем тип вопроса **text** с уточнением что в качестве ответа должно быть предоставлено одно слово **restriction=word**.



```
{
  "id": "yandex_tutor_T7635",
  "meta": {
    "category": "ege_russian",
    "language": "ru"
  },
  "score": 1,
  "text": "В одном из приведённых ниже предложений **неверно** употреблено выделенное слово. Исправьте лексическую ошибку, подобрав к выделенному слову пароним.\n\n- Работа молодого актёра небезупречна, однако в силу его склонности к созданию ЦЕЛОСТНОГО образа ему удалось подчеркнуть способность героя естественно откликаться на просьбы окружающих.\n- Пьеса довольно СЦЕНИЧНА и прекрасно сыграна.\n- Маятник ОТКЛОНИЛСЯ примерно на шестьдесят градусов.\n- Наше основное РАЗЛИЧИЕ друг от друга состоит не во внешности, а в восприятии мира как такового.\n- На ближайший месяц утверждён новый ПРОИЗВОДСТВЕННЫЙ план.\n\nЗапишите подобранное слово.",
  "attachments": [],
  "question": {
    "type": "text",
    "restriction": "word"
  },
  "solution": {
    "correct": "отличие",
    "text": "**Указание:** Лексические нормы.\n\n**Решение:** Вместо слова различие в данном контексте необходимо употребить слово отличие, поскольку далее используется предлог от (отличие чего-либо от чего либо)."
  }
}
```



В случае нескольких допустимых вариантов ответа необходимо использовать поле **correct_variants**. Например:


```
{
  "solution": {
    "correct_variants": ["которой", "который"]
  }
}
```



In [0]:
from ufal.udpipe import Model, Pipeline
from difflib import get_close_matches
from string import punctuation
import pickle
import pymorphy2
import re
import random


class Solver5(object):

    def __init__(self, seed=42):

        self.morph = pymorphy2.MorphAnalyzer()
        self.model = Model.load("data/udpipe_syntagrus.model")#.encode()
        self.process_pipeline = Pipeline(self.model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
        self.seed = seed
        self.init_seed()
        self.paronyms = self.get_paronyms()
        self.freq_bigrams = self.open_freq_grams()

    def init_seed(self):
        return random.seed(self.seed)

    def open_freq_grams(selfself):
        with open('data/bigrams_lemmas.pickle', 'rb') as inputfile:
            counts = pickle.load(inputfile)
        return counts

# Паронимы - это слова, близкие по звучанию, но различающиеся частично или полностью значением.
    def get_paronyms(self):
        paronyms = []
        with open('data/paronyms.csv', 'r', encoding='utf-8') as in_file:
            for line in in_file.readlines():
                pair = line.strip(punctuation).split('\t')
                paronyms.append(pair)
        return paronyms

    def lemmatize(self, token):
        token_all = self.morph.parse(token.lower().rstrip('.,/;!:?'))[0]
        lemma = token_all.normal_form
        return lemma

    def find_closest_paronym(self, par):
        paronyms = set()
        for par1, par2 in self.paronyms:
            paronyms.add(par1)
            paronyms.add(par2)
        try:
            closest = get_close_matches(par, list(paronyms))[0]
        except IndexError:
            closest = None
        return closest

    def check_pair(self, token_norm):
        paronym = None
        for p1, p2 in self.paronyms:
            if token_norm == p1:
                paronym = p2
                break
            if token_norm == p2:
                paronym = p1
                break
        return paronym

    def find_paronyms(self, token):
        token_all = self.morph.parse(token.lower().rstrip('.,/;!:?'))[0]
        token_norm = token_all.normal_form
        paronym = self.check_pair(token_norm)

        if paronym is None:
            paronym_close = self.find_closest_paronym(token_norm)
            paronym = self.check_pair(paronym_close)

        if paronym is not None:
            paronym_parse = self.morph.parse(paronym)[0]
            try:
                str_grammar = str(token_all.tag).split()[1]
            except IndexError:
                str_grammar = str(token_all.tag)

            gr = set(str_grammar.replace("Qual ", "").replace(' ',',').split(','))
            try:
                final_paronym = paronym_parse.inflect(gr).word
            except AttributeError:
                final_paronym = paronym
        else:
            final_paronym = ''
        return final_paronym

    def syntax_parse(self, some_text, token):
        processed = self.process_pipeline.process(some_text.lower())
        content = [l for l in processed.split('\n') if not l.startswith('#')]
        tagged = [w.split('\t') for w in content if w]

        linked_word = ''
        for analysis in tagged:
            if analysis[1] == token:
                head = analysis[6]
                if head == '0':
                    root_id = analysis[0]
                    for analysis in tagged:
                        if analysis[6] == root_id:
                            linked_word = analysis[1]
                            break
                else:
                    for analysis in tagged:
                        if analysis[0] == head:
                            linked_word = analysis[1]
                            break
        return linked_word

    def check_frequencies(self, sentences):
        examples = []
        for token, second_tok, line in sentences:
            token = token.lower().rstrip('.,;:!?')
            token_lemma = self.lemmatize(token)
            second_lemma = self.lemmatize(second_tok)

            collocation_word = self.syntax_parse(line, token)
            collocation_lemma = self.lemmatize(collocation_word)

            first = (collocation_lemma, token_lemma)
            second = (collocation_lemma, second_lemma)

            freq1 = self.freq_bigrams[first]
            freq2 = self.freq_bigrams[second]

            first = (token_lemma, collocation_lemma)
            second = (second_lemma, collocation_lemma)
            freq3 = self.freq_bigrams[first]
            freq4 = self.freq_bigrams[second]

            freq_first = freq1 + freq3
            freq_second = freq2 + freq4

            if freq_second > freq_first:
                return second_tok
            if freq_first == freq_second:
                examples.append((0, freq_first, freq_second, token, second_tok))

        good_paronym = ''
        if examples:
            good_paronym = examples[0][4]
        return good_paronym

    def predict(self, task):
        return self.predict_from_model(task)

    def fit(self, tasks):
        pass
        
    def load(self, path="data/models/solver5.pkl"):
        pass

    def save(self, path="data/models/solver5.pkl"):
        pass

    def predict_from_model(self, task):
        description = task["text"].replace('НЕВЕРНО ', "неверно ")
        sents = []
        for line in description.split("\n"):
            for token in line.split():
                if token.isupper() and len(token) > 2: # get CAPS paronyms
                    second_pair = self.find_paronyms(token)
                    sents.append((token, second_pair, line))
        result = self.check_frequencies(sents)
        return result.strip(punctuation+'\n')

Solver 6

#### В формате JSON


```
[{'attachments': [],
  'id': '6',
  'question': {'max_length': 30,
   'recommended_length': 20,
   'restriction': 'word',
   'type': 'text'},
  'score': 1,
  'solution': {'correct': 'коренные'},
  'text': 'Отредактируйте предложение: исправьте лексическую ошибку, исключив
   лишнее слово. Выпишите это слово (если с ним находится предлог, выписывать
    нужно вместе с предлогом).\nПодобное оружие делали и коренные аборигены 
    Австралии, только они закрепляли зубы на дубинке не жгутом, а воском, 
    вырабатываемым особыми пчёлами, которые не имели жала.'}]
```




In [0]:
import random
import re
import nltk
import pymorphy2
from nltk.util import ngrams
from sklearn.metrics.pairwise import cosine_similarity
from solvers.utils import BertEmbedder
from string import punctuation


class Solver6(BertEmbedder):

    def __init__(self, seed=42):
        super(Solver6, self).__init__()
        self.seed = seed
        self.init_seed()
        self.morph = pymorphy2.MorphAnalyzer()
        self.has_model = True
        self.mode = 1 # 1 - find wrong word, 2 - replace word

    def init_seed(self):
        return random.seed(self.seed)

    def predict_random(self, task_desc):
        """Random variant"""
        task_desc = re.sub("[^а-я0-9\-]", " ", task_desc)
        result = random.choice(task_desc.split())
        return result

    def exclude_word(self, task_sent):
        """Make it with Bert"""
        tokens = [token.strip('.,";!:?><)«»') for token in task_sent.split(" ") if token != ""]

        to_tokens = []
        for token in tokens:
            parse_res = self.morph.parse(token)[0]
            if parse_res.tag.POS not in ["CONJ", "PREP", "PRCL", "INTJ", "PRED", "NPRO"]:
                if parse_res.normal_form != 'быть':
                    to_tokens.append((parse_res.word, parse_res.tag.POS))

        bigrams = list(ngrams(to_tokens, 2))
        print(bigrams)

        results = []
        for bigram in bigrams:
            if bigram[0] != bigram[1]:
                b1 = self.sentence_embedding([bigram[0][0]])[0].reshape(1, -1)
                b2 = self.sentence_embedding([bigram[1][0]])[0].reshape(1, -1)
                sim = cosine_similarity(b1, b2)[0][0]
                results.append((sim, bigram[0][0], bigram[1][0], bigram[0][1], bigram[0][1]))
        results = sorted(results)
        final_pair = results[-1]
        if final_pair[-1] == 'NOUN' and final_pair[-2] == 'NOUN':
            print (results[-1][2], tokens)
            return results[-1][2], tokens
        else:
            print (results[-1][1], tokens)
            return results[-1][1], tokens

    def fit(self, tasks):
        pass
        
    def load(self, path="data/models/solver6.pkl"):
        pass

    def save(self, path="data/models/solver6.pkl"):
        pass

    def predict(self, task):
        if not self.has_model:
            return self.predict_random(task)
        else:
            return self.predict_from_model(task)

    def predict_from_model(self, task):
        description = task["text"]
        task_desc = ""
        if "заменив" in description:
            self.mode = 2
        else:
            self.mode = 1
        for par in description.split("\n"):
            for sentence in nltk.sent_tokenize(par):
                sentence = sentence.lower().rstrip(punctuation).replace('6.', "")
                if re.match('.*(отредактируйте|выпишите|запишите|исправьте|исключите).*', sentence):
                    continue
                else:
                    task_desc += sentence
        result, tokens = self.exclude_word(task_desc)
        return result.strip(punctuation)


Solver 8

#### Установка соответствия

#### Text
Установите соответствие между грамматическими ошибками и предложениями, в которых они допущены: к каждой позиции первого столбца подберите соответствующую позицию из второго столбца.

#### Question
ГРАММАТИЧЕСКИЕ ОШИБКИ

A. нарушение в построении предложения с несогласованным приложением B. нарушение видо-временной соотнесённости глагольных форм C. нарушение в построении предложения с деепричастным оборотом D. нарушение в построении предложения с косвенной речью E. неправильное употребление падежной формы существительного с предлогом.

ПРЕДЛОЖЕНИЯ



1.   Масаока Сики считал, что «короткие стихи не могут передать течение времени, поэтому поэт-хайкаист изображает не время, а лишь пространство».
2.   Отец заглянул в комнату и, стараясь говорить не слишком громко, чтобы не испугать погружённого в работу Костика, спросил, что «не хочешь ли ты есть»?
3.   Вопреки распространённого мнения, лучшие рестораны Франции находятся вовсе не в Париже.
4.   В прошлом номере газеты «Грани таланта» опубликована рецензия на новую книгу известного писателя, и автор этой рецензии, не жалея красивых фраз, даёт роману очень высокую оценку.
5.   В статье «Вы и ваши дети» психолог стремится привлечь внимание родителей к теме воспитания детей; для этого он рассказывал об интересных случаях из собственной практики.
6.   Мы с приятелем сидели в нашей комнате в общежитии, когда он, неожиданно для меня, сказал, что «разочарован в своей идее», которую вынашивал много лет.
7.   Заканчивая свою речь, хочу вспомнить слова одного философа, который утверждал, что «человек всегда умирает до того, как полностью родится».
8.   В очередном выпуске журнала «Науки и жизни» есть очень увлекательная история, рассказывающая о том, как живут жирафы.
9.   Для человека, даже находясь в состоянии угнетения, тоски и грусти, очень важна вера в светлое будущее, в то, что всё непременно будет хорошо.



### В формате JSON



```
# This is formatted as code{
  "id": "yandex_tutor_T7656",
  "meta": {
    "category": "ege_russian",
    "language": "ru"
  },
  "score": 3,
  "text": "Установите соответствие между грамматическими ошибками и предложениями, в которых они допущены: к каждой позиции первого столбца подберите соответствующую позицию из второго столбца.",
  "attachments": [],
  "question": {
    "type": "matching",
    "left": [
      {"id": "A", "text": "нарушение в построении предложения с несогласованным приложением"},
      {"id": "B", "text": "нарушение видо-временной соотнесённости глагольных форм"},
      {"id": "C", "text": "нарушение в построении предложения с деепричастным оборотом"},
      {"id": "D", "text": "нарушение в построении предложения с косвенной речью"},
      {"id": "E", "text": "неправильное употребление падежной формы существительного с предлогом"}
    ],
    "choices": [
      {"id": "1", "text": "Масаока Сики считал, что «короткие стихи не могут передать течение времени, поэтому поэт-хайкаист изображает не время, а лишь пространство»."},
      {"id": "2", "text": "Отец заглянул в комнату и, стараясь говорить не слишком громко, чтобы не испугать погружённого в работу Костика, спросил, что «не хочешь ли ты есть»?"},
      {"id": "3", "text": "Вопреки распространённого мнения, лучшие рестораны Франции находятся вовсе не в Париже."},
      {"id": "4", "text": "В прошлом номере газеты «Грани таланта» опубликована рецензия на новую книгу известного писателя, и автор этой рецензии, не жалея красивых фраз, даёт роману очень высокую оценку."},
      {"id": "5", "text": "В статье «Вы и ваши дети» психолог стремится привлечь внимание родителей к теме воспитания детей; для этого он рассказывал об интересных случаях из собственной практики."},
      {"id": "6", "text": "Мы с приятелем сидели в нашей комнате в общежитии, когда он, неожиданно для меня, сказал, что «разочарован в своей идее», которую вынашивал много лет."},
      {"id": "7", "text": "Заканчивая свою речь, хочу вспомнить слова одного философа, который утверждал, что «человек всегда умирает до того, как полностью родится»."},
      {"id": "8", "text": "В очередном выпуске журнала «Науки и жизни» есть очень увлекательная история, рассказывающая о том, как живут жирафы."},
      {"id": "9", "text": "Для человека, даже находясь в состоянии угнетения, тоски и грусти, очень важна вера в светлое будущее, в то, что всё непременно будет хорошо."}
    ]
  },
  "solution": {
    "correct": {
      "A": "8",
      "B": "5",
      "C": "9",
      "D": "2",
      "E": "3"
    }
  }
}
```



In [0]:
from ufal.udpipe import Model, Pipeline
from difflib import SequenceMatcher
from string import punctuation
import pymorphy2
import random
import re
import sys


def get_gerund(features):
    """деепричастие """
    hypothesys = []

    for feature in features:
        for row in feature:
            if row[4] == "VERB":
                if "VerbForm=Conv" in row[5]:
                    hypothesys.append(" ".join([row[2] for row in feature]))

    return hypothesys

def get_indirect_speech(features):
    """ косвенная речь """
    hypothesys = []
    for feature in features:
        for row in feature:
            if row[8] == '1':
                hypothesys.append(" ".join([row[2] for row in feature]))
    return hypothesys

def get_app(features):
    """ Приложение """
    hypothesys = []
    for feature in features:
        for row1, row2, row3 in zip(feature, feature[1:], feature[2:]):
            if row1[2] == "«" and row3[2] == "»" and row2[1] == '1':
                hypothesys.append(" ".join([row[2] for row in feature]))
            if "«" in row1[2]:
                if row1[2][1:][0].isupper():
                    hypothesys.append(" ".join([row[2] for row in feature]))
    return hypothesys

def get_predicates(features):
    """ связь подлежащее сказуемое root + subj = number """
    hypothesys = set()

    for feature in features:
        head, number = None, None
        for row in feature:
            if row[7] == 'root':
                head = row[0]
                for s in row[5].split('|'):
                    if "Number" in s:
                        number = s.replace("Number=", "")
        for row in feature:
            row_number = None
            for s in row[5].split('|'):
                if "Number" in s:
                    row_number = s.replace("Number=", "")
            if row[0] == head and number != row_number:
                hypothesys.add(" ".join([row[2] for row in feature]))
    return hypothesys

def get_clause(features):
    """ сложные предложения """
    hypothesys = set()
    for feature in features:
        for row in feature:
            if row[3] == 'который':
                hypothesys.add(" ".join([row[2] for row in feature]))
    return hypothesys


def get_participle(features):
    """причастие """
    hypothesys = []
    for feature in features:
        for row in feature:
            if row[4] == "VERB":
                if "VerbForm=Part" in row[5]:
                    hypothesys.append(" ".join([row[2] for row in feature]))
    return hypothesys

def get_verbs(features):
    """ вид и время глаголов """
    hypothesys = set()
    for feature in features:
        head, aspect, tense = None, None, None
        for row in feature:
            if row[7] == 'root':
                # head = row[0]
                for s in row[5].split('|'):
                    if "Aspect" in s:
                        aspect = s.replace("Aspect=", "")
                    if "Tense" in s:
                        tense = s.replace("Tense=", "")

        for row in feature:
            row_aspect, row_tense = None, None
            for s in row[5].split('|'):
                if "Aspect" in s:
                    row_aspect = s.replace("Aspect=", "")
            for s in row[5].split('|'):
                if "Tense" in s:
                    row_tense = s.replace("Tense=", "")
            if row[4] == "VERB" and row_aspect != aspect: # head ?
                hypothesys.add(" ".join([row[2] for row in feature]))

            if row[4] == "VERB" and row_tense != tense:
                hypothesys.add(" ".join([row[2] for row in feature]))
    return hypothesys

def get_nouns(features):
    """ формы существительных ADP + NOUN"""
    hypothesys = set()
    apds = ["благодаря", "согласно", "вопреки", "подобно", "наперекор",
            "наперерез", "ввиду", "вместе", "наряду", "по"]
    for feature in features:
        for row1, row2 in zip(feature, feature[1:]):
            if row1[3] in apds:
                if row2[4] == 'NOUN':
                    hypothesys.add(" ".join([row[2] for row in feature]))
    return hypothesys

def get_numerals(features):
    hypothesys = []
    for feature in features:
            for row in feature:
                if row[4] == "NUM":
                    hypothesys.append(" ".join([row[2] for row in feature]))
    return hypothesys


def get_homogeneous(features):
    hypothesys = set()
    for feature in features:
        sent = " ".join([token[2] for token in feature]).lower()
        for double_conj in ["если не", "не столько", "не то чтобы"]:
            if double_conj in sent:
                hypothesys.add(sent)
    return hypothesys


class Solver8():

    def __init__(self, seed=42):
        self.morph = pymorphy2.MorphAnalyzer()
        self.categories = set()
        self.has_model = True
        self.model = Model.load("data/udpipe_syntagrus.model")
        self.process_pipeline = Pipeline(self.model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
        self.seed = seed
        self.label_dict = {
            'деепричастный оборот': "get_gerund",
            'косвенный речь': "get_indirect_speech",
            'несогласованный приложение': "get_app",
            'однородный член': "get_homogeneous",
            'причастный оборот': "get_participle",
            'связь подлежащее сказуемое': "get_predicates",
            'сложноподчинённый': "get_clause",
            'сложный': "get_clause",
            'соотнесённость глагольный форма': "get_verbs",
            'форма существительное': "get_nouns",
            'числительное': "get_numerals"
        }
        self.init_seed()

    def init_seed(self):
        return random.seed(self.seed)

    def get_syntax(self, text):
        processed = self.process_pipeline.process(text)
        content = [l for l in processed.split('\n') if not l.startswith('#')]
        tagged = [w.split('\t') for w in content if w]
        return tagged

    def tokens_features(self, some_sent):

        tagged = self.get_syntax(some_sent)
        features = []
        for token in tagged:
            _id, token, lemma, pos, _, grammar, head, synt, _, _, = token #tagged[n]
            capital, say = "0", "0"
            if lemma[0].isupper():
                    capital = "1"
            if lemma in ["сказать", "рассказать", "спросить", "говорить"]:
                    say = "1"
            feature_string = [_id, capital, token, lemma, pos, grammar, head, synt, say]
            features.append(feature_string)
        return features

    def normalize_category(self, cond):
        """ {'id': 'A', 'text': 'ошибка в построении сложного предложения'} """
        condition = cond["text"].lower().strip(punctuation)
        condition = re.sub("[a-дabв]\)\s", "", condition).replace('членами.', "член")
        norm_cat = ""
        for token in condition.split():
            lemma = self.morph.parse(token)[0].normal_form
            if lemma not in [
                    "неправильный", "построение", "предложение", "с", "ошибка", "имя",
                    "видовременной", "видо-временной", "предложно-падежный", "падежный",
                    "неверный", "выбор", "между", "нарушение", "в", "и", "употребление",
                    "предлог", "видовременный", "временной"
                ]:
                norm_cat += lemma + ' '
        self.categories.add(norm_cat[:-1])
        return norm_cat

    def parse_task(self, task):

        assert task["question"]["type"] == "matching"

        conditions = task["question"]["left"]
        choices = task["question"]["choices"]

        good_conditions = []
        X = []
        for cond in conditions:  # LEFT
            good_conditions.append(self.normalize_category(cond))
                    
        for choice in choices:
            choice = re.sub("[0-9]\\s?\)", "", choice["text"])
            X.append(choice)
        return X, choices, good_conditions

    def match_choices(self, label2hypothesys, choices):
        final_pred_dict = {}
        for key, value in label2hypothesys.items():
            if len(value) == 1:
                variant = list(value)[0]
                variant = variant.replace(' ,', ',')
                for choice in choices:
                    ratio = SequenceMatcher(None, variant, choice["text"]).ratio()
                    if ratio > 0.9:
                        final_pred_dict[key] = choice["id"]
                        choices.remove(choice)

        for key, value in label2hypothesys.items():
            if key not in final_pred_dict.keys():
                variant = []
                for var in value:
                    for choice in choices:
                        ratio = SequenceMatcher(None, var, choice["text"]).ratio()
                        if ratio > 0.9:
                            variant.append(var)
                if variant:
                    for choice in choices:
                        ratio = SequenceMatcher(None, variant[0], choice["text"]).ratio()
                        if ratio > 0.9:
                            final_pred_dict[key] = choice["id"]
                            choices.remove(choice)
                else:
                    variant = [choice for choice in choices]
                    if variant:
                        final_pred_dict[key] = variant[0]["id"]
                        for choice in choices:
                            ratio = SequenceMatcher(None, variant[0]["text"], choice["text"]).ratio()
                            if ratio > 0.9:
                                choices.remove(choice)

        for key, value in label2hypothesys.items():
            if key not in final_pred_dict.keys():
                variant = [choice for choice in choices]
                if variant:
                    final_pred_dict[key] = variant[0]["id"]
                    for choice in choices:
                        ratio = SequenceMatcher(None, variant[0]["text"], choice["text"]).ratio()
                        if ratio > 0.9:
                            choices.remove(choice)
        return final_pred_dict

    def predict_random(self, task):
        """ Test a random choice model """
        conditions = task["question"]["left"]
        choices = task["question"]["choices"]
        pred = {}
        for cond in conditions:
            pred[cond["id"]] = random.choice(choices)["id"]
        return pred

    def predict(self, task):
        if not self.has_model:
            return self.predict_random(task)
        else:
            return self.predict_from_model(task)

    def fit(self, tasks):
        pass

    def load(self, path="data/models/solver8.pkl"):
        pass

    def save(self, path="data/models/solver8.pkl"):
        pass

    def predict_from_model(self, task):
        x, choices, conditions = self.parse_task(task)
        all_features = []
        for row in x:
            all_features.append(self.tokens_features(row))

        label2hypothesys = {}
        for label in self.label_dict.keys():
            func = self.label_dict[label.rstrip()]
            hypotesis = getattr(sys.modules[__name__], func)(all_features)
            label2hypothesys[label] = hypotesis

        final_pred_dict = self.match_choices(label2hypothesys, choices)
        
        pred_dict = {}
        for cond, key in zip(conditions, ["A", "B", "C", "D", "E"]):
            cond = cond.rstrip()
            try:
                pred_dict[key] = final_pred_dict[cond]
            except KeyError:
                pred_dict[key] = "1"
        return pred_dict

Solver 9

#### В формате JSON

```
[{'attachments': [],
  'id': '9',
  'question': {'choices': [{'id': '1',
     'text': 'раск..рмить, раск..лить, раск..лоть'},
    {'id': '2', 'text': 'созд..вать, разж..гать, ш..повник'},
    {'id': '3', 'text': 'благосл..вение, пок..яние, нав..ждение'},
    {'id': '4', 'text': 'вопл..тить, отпл..тить, упл..тнить'},
    {'id': '5', 'text': 'подб..родок, оз..рить, импр..визировать'}],
   'min_choices': 1,
   'type': 'multiple_choice'},
  'score': 1,
  'solution': {'correct_variants': [['1', '4'], ['4', '1']]},
  'text': 'Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.Запишите номера ответов.'}]
```



In [0]:
import re
import random
from solvers.utils import standardize_task, AbstractSolver


class Solver9(AbstractSolver):
    def __init__(self, **kwargs):
        self.known_examples = {
            "alternations": ["г..р", "з..р", "к..с", "кл..н",  "л..г", "л..ж", "м..к", "пл..в",
                             "р..вн", "р..с", "р..ст", "р..щ", "ск..к", "ск..ч", "тв..р", "б..р",
                             "д..р", "м..р", "п..р", "т..р", "бл..ст", "ж..г", "ст..л", "ч..т"],
            "verifiable": [],
            "unverifiable": []
        }

        self.exceptions = {
            "alternations": ["ст..лист", "прим..р", "г..рева", "г..рю", "г..рд", "алг..ритм", "г..ризонт", "г..рист"],
            "verifiable": [],
            "unverifiable": []
        }
        super().__init__()

    def predict_from_model(self, task):
        task = standardize_task(task)
        text, choices = task["text"], task["question"]["choices"]
        alt, unver = "чередующаяся", "непроверяемая"
        type_ = "alternations" if alt in text else "unverifiable" if unver in text else "verifiable"
        nice_option_ids = list()
        for option in choices:
            parsed_option = re.sub(r"^\d\)", "", option["text"]).split(", ")
            if all(self.is_of_type(word, type_) for word in parsed_option):
                nice_option_ids.append(option["id"])
        if choices[0]["text"].count(", ") == 0:
            if len(nice_option_ids) == 0:
                return [random.choice([str(i + 1) for i in range(5)])]
            elif len(nice_option_ids) == 1:
                return nice_option_ids
            else:
                return [random.choice(nice_option_ids)]
        else:
            if len(nice_option_ids) == 0:
                return sorted(random.sample([str(i + 1) for i in range(5)], 2))
            elif len(nice_option_ids) == 1:
                return sorted(nice_option_ids + [random.choice([str(i + 1) for i in range(5)
                                                                if str(i + 1) != nice_option_ids[0]])])
            elif len(nice_option_ids) in [2, 3]:
                return sorted(nice_option_ids)
            else:
                return sorted(random.sample(nice_option_ids, 2))

    def fit(self, tasks):
        alt, unver = "чередующаяся", "непроверяемая"
        for task in tasks:
            task = standardize_task(task)
            text = task["text"]

            if alt in text:
                type_ = "alternations"
            elif unver in text:
                type_ = "unverifiable"
            else:
                type_ = "verifiable"

            correct = task["solution"]["correct_variants"][0] if "correct_variants" in task["solution"] \
                else task["solution"]["correct"]
            for correct_id in correct:
                for word in task["choices"][int(correct_id) - 1]["parts"]:
                    word_sub = re.sub(r" *(?:^\d\)|\(.*?\)) *", "", word)
                    self.known_examples[type_].append(word_sub)
        return self

    def is_of_type(self, word, type_):
        if any(alternation in word for alternation in self.known_examples[type_]) \
                and not any(exception in word for exception in self.exceptions):
            return True
        else:
            return False


Solver 17

#### В формате JSON

```
[{'attachments': [],
  'id': '17',
  'question': {'choices': [{'id': '1', 'placeholder': '(1)'},
    {'id': '2', 'placeholder': '(2)'},
    {'id': '3', 'placeholder': '(3)'},
    {'id': '4', 'placeholder': '(4)'},
    {'id': '5', 'placeholder': '(5)'},
    {'id': '6', 'placeholder': '(6)'},
    {'id': '7', 'placeholder': '(7)'}],
   'min_choices': 1,
   'type': 'multiple_choice'},
  'score': 1,
  'solution': {'correct': ['1', '2', '3']},
  'text': 'Расставьте знаки препинания: укажите цифру(-ы), на месте которой(-ых)
   в предложении должна(-ы) стоять запятая(-ые).\nВысокие, узкие клочья тумана 
   (1) густые и белые (2) бродили над рекой (3) заслоняя (4) отражение звёзд (5)
    и (6) цепляясь (7) за ивы.'}]
```



In [0]:
import re
import pymorphy2
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from solvers.utils import AbstractSolver
import joblib


class Solver17(AbstractSolver):
    def __init__(self, seed=42, train_size=0.85):
        self.has_model = False
        self.is_train_task = False
        self.morph = pymorphy2.MorphAnalyzer()
        self.pos2n = {None: 0}
        self.n2pos = [None, ]
        self.train_size = train_size
        self.seed = seed
        self.model = CatBoostClassifier(loss_function="Logloss",
                                   eval_metric='Accuracy',
                                   use_best_model=True, random_seed=self.seed)
        super().__init__(seed)

    def get_placeholder(self, token):
        if len(token) < 5 and token[0] == '(' and token[-1] == ')':
            return token[1:-1]
        return ''

    def save(self, path="data/models/solver17.pkl"):
        joblib.dump(self.model, path)

    def load(self, path="data/models/solver17.pkl"):
        self.model = joblib.load(path)

    def get_target(self, task):
        if 'solution' not in task:
            return []
        y_true = task['solution']['correct_variants'] if 'correct_variants' in task['solution'] \
            else [task['solution']['correct']]
        return list(y_true[0])

    def clear_token(self, token):
        for char in '?.!/;:':
            token = token.replace(char, '')
        return token

    def get_feat(self, token):
        if self.get_placeholder(token):
            return 'PHDR'
        else:
            p = self.morph.parse(self.clear_token(token))[0]
            return str(p.tag.POS)

    def encode_feats(self, feats):
        res = []
        for feat in feats:
            if feat not in self.pos2n and self.is_train_task:
                self.pos2n[feat] = len(self.pos2n)
                self.n2pos.append(feat)
            elif feat not in self.pos2n:
                feat = None
            res.append(self.pos2n[feat])
        return res

    def correct_spaces(self, text):
        text = re.sub(r'(\(\d\))', r' \1 ', text)
        text = re.sub(r'  *', r' ', text)
        return text

    def parse_task(self, task):
        feat_ids = [-3, -2, -1, 1, 2, 3]
        tokens = self.correct_spaces(task['text']).split()
        targets = self.get_target(task)
        X, y = [], []
        for i, token in enumerate(tokens):
            placeholder = self.get_placeholder(token)
            if not placeholder:
                continue
            if placeholder in targets:
                y.append(1)
            else:
                y.append(0)
            feats = []
            for feat_idx in feat_ids:
                if i + feat_idx < 0 or i + feat_idx >= len(tokens):
                    feats.append('PAD')
                else:
                    feats.append(self.get_feat(tokens[i + feat_idx]))
            X.append(self.encode_feats(feats))
        return X, y

    def fit(self, tasks):
        self.is_train_task = True
        X, y = [], []
        for task in tasks:
            task_x, task_y = self.parse_task(task)
            X += task_x
            y += task_y
        X_train, X_dev, Y_train, Y_dev = train_test_split(X, y, shuffle=True,
                                                          train_size=self.train_size, random_state=self.seed)
        cat_features = [0, 1, 2, 3, 4, 5]
        self.model = self.model.fit(X_train, Y_train, cat_features, eval_set=(X_dev, Y_dev))
        self.has_model = True

    def predict_from_model(self, task):
        self.is_train_task = False
        X, y = self.parse_task(task)
        pred = self.model.predict(X)
        pred = [str(i+1) for i, p in enumerate(pred) if p >= 0.5]
        return pred if pred else ["1"]


In [0]:
solver17 = Solver17(train_size=0.9)

Solver 21

### Ссылка на предложение

**Text**
Найдите предложения, в которых тире ставится в соответствии с одним и тем же правилом пунктуации.


> (1) Севернее города Одинцово раскинулся Подушкинский лесопарк – живописная лесная территория. (2) Этот лес получил своё название благодаря селу Подушкино и лично Ивану Владимировичу Подушке – помещику, которому принадлежали эти земли в XV веке.
> (3) Село Подушкино и прилегающие к нему леса не раз меняли хозяев: так, одно время этими угодьями владел Илья Данилович Милославский, дед царевны Софьи и двух царей – Фёдора III и Ивана V. (4) А в XIX веке в Подушкино провели железную дорогу – здесь появились дачники.
> (5) Сейчас, прогуливаясь по парку, можно увидеть двухсотлетние дубы, однако бóльшая часть сохранившихся на сегодня деревьев появилась 50 – 80 лет назад. (6) По оценкам биологов, в Подушкинском лесу обитает 73 вида животных, причём помимо привычных белок и зайцев в лесной глуши можно увидеть норы барсука и встретить орешниковую соню – небольшого грызуна, занесённого в Красную книгу. (7) Зелёные дятлы, глухари, рябчики, чёрные коршуны и сапсаны – эти редкие птицы также обитают на территории Подушкинского лесопарка.
> (По материалам интернета)











Question

Необходимо выбрать правильное подмножество предложений из текста.

#### В формате JSON

Аналогично предыдущему примеру используем тип вопроса **multiple_choice**, в котором в вариантах ответа указаны ссылки на места в тексте. Ссылки на места в тексте указываются при помощи поля **link**, имеющего похожий на **placeholder** смысл — указатель на уникальную подстроку в тексте. В отличие от **placeholder**, **link** не имеет в виду что в тексте пропущена информация, а имеет смысл указателя на опеределенное место в тексте (в данном задании — указатель на предложение).



```
{
  "id": "yandex_tutor_T7651",
  "meta": {
    "category": "ege_russian",
    "language": "ru"
  },
  "score": 2,
  "text": "Найдите предложения, в которых **тире** ставится в соответствии с одним и тем же правилом пунктуации.\n\n> (1) Севернее города Одинцово раскинулся Подушкинский лесопарк – живописная лесная территория. (2) Этот лес получил своё название благодаря селу Подушкино и лично Ивану Владимировичу Подушке – помещику, которому принадлежали эти земли в XV веке.\n> \n> (3) Село Подушкино и прилегающие к нему леса не раз меняли хозяев: так, одно время этими угодьями владел Илья Данилович Милославский, дед царевны Софьи и двух царей – Фёдора III и Ивана V. (4) А в XIX веке в Подушкино провели железную дорогу – здесь появились дачники.\n> \n> (5) Сейчас, прогуливаясь по парку, можно увидеть двухсотлетние дубы, однако бóльшая часть сохранившихся на сегодня деревьев появилась 50 – 80 лет назад. (6) По оценкам биологов, в Подушкинском лесу обитает 73 вида животных, причём помимо привычных белок и зайцев в лесной глуши можно увидеть норы барсука и встретить орешниковую соню – небольшого грызуна, занесённого в Красную книгу. (7) Зелёные дятлы, глухари, рябчики, чёрные коршуны и сапсаны – эти редкие птицы также обитают на территории Подушкинского лесопарка.\n> \n> (По материалам интернета)",
  "attachments": [],
  "question": {
    "type": "multiple_choice",
    "choices": [
      {"id": "1", "link": "(1)"},
      {"id": "2", "link": "(2)"},
      {"id": "3", "link": "(3)"},
      {"id": "4", "link": "(4)"},
      {"id": "5", "link": "(5)"},
      {"id": "6", "link": "(6)"},
      {"id": "7", "link": "(7)"}
    ],
    "min_choices": 1
  },
  "solution": {
    "correct": ["1", "2", "3", "6"]
  }
}
```



In [39]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.models import Model
from itertools import combinations
from keras.preprocessing.sequence import pad_sequences
from solvers.utils import BertEmbedder, singleton
import re
import numpy as np
import random
import gensim
import time
import os


class SiameseBiLSTM(BertEmbedder):

    def __init__(self, model=None, embedding_dim=768, max_sequence_length=40, number_lstm=50, number_dense=50, rate_drop_lstm=0.17,
                 rate_drop_dense=0.25, hidden_activation='relu', validation_split_ratio=0.2):
        super(SiameseBiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.max_sequence_length = max_sequence_length
        self.number_lstm_units = number_lstm
        self.rate_drop_lstm = rate_drop_lstm
        self.number_dense_units = number_dense
        self.activation_function = hidden_activation
        self.rate_drop_dense = rate_drop_dense
        self.validation_split_ratio = validation_split_ratio

    def train_model(self, sentences_pairs, is_similar, model_save_directory='./'):
        """
        Train Siamese network to find similarity between sentences in `sentences_pair`
            Steps Involved:
                1. Pass the each from sentences_pairs  to bidirectional LSTM encoder.
                2. Merge the vectors from LSTM encodes and passed to dense layer.
                3. Pass the  dense layer vectors to sigmoid output layer.
                4. Use cross entropy loss to train weights
        Args:
            sentences_pair (list): list of tuple of sentence pairs
            is_similar (list): target value 1 if same sentences pair are similar otherwise 0
        Returns:
            return (best_model_path):  path of best model
        """
        train_data_x1, train_data_x2, train_labels, leaks_train, \
        val_data_x1, val_data_x2, val_labels, leaks_val = self.create_train_dev_set(
            sentences_pairs, is_similar, self.max_sequence_length,
            self.validation_split_ratio
        )

        embedding_layer = Embedding(119547, 768, weights=[self.embedding_matrix], trainable=False)

        lstm_layer = Bidirectional(
            LSTM(self.number_lstm_units, dropout=self.rate_drop_lstm, recurrent_dropout=self.rate_drop_lstm)
        )

        sequence_1_input = Input(shape=(self.max_sequence_length,), dtype='int32')
        embedded_sequences_1 = embedding_layer(sequence_1_input)
        x1 = lstm_layer(embedded_sequences_1)

        sequence_2_input = Input(shape=(self.max_sequence_length,), dtype='int32')
        embedded_sequences_2 = embedding_layer(sequence_2_input)
        x2 = lstm_layer(embedded_sequences_2)

        leaks_input = Input(shape=(leaks_train.shape[1],))
        leaks_dense = Dense(int(self.number_dense_units/2), activation=self.activation_function)(leaks_input)

        merged = concatenate([x1, x2, leaks_dense])
        merged = BatchNormalization()(merged)
        merged = Dropout(self.rate_drop_dense)(merged)
        merged = Dense(self.number_dense_units, activation=self.activation_function)(merged)
        merged = BatchNormalization()(merged)
        merged = Dropout(self.rate_drop_dense)(merged)
        preds = Dense(1, activation='sigmoid')(merged)

        model = Model(inputs=[sequence_1_input, sequence_2_input, leaks_input], outputs=preds)
        model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=3)

        STAMP = 'lstm_%d_%d_%.2f_%.2f' % (self.number_lstm_units, self.number_dense_units, self.rate_drop_lstm, self.rate_drop_dense)

        checkpoint_dir = model_save_directory + 'checkpoints/' + str(int(time.time())) + '/'

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        bst_model_path = checkpoint_dir + STAMP + '.h5'

        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)

        tensorboard = TensorBoard(log_dir=checkpoint_dir + "logs/{}".format(time.time()))

        model.fit([train_data_x1, train_data_x2, leaks_train], train_labels,
                  validation_data=([val_data_x1, val_data_x2, leaks_val], val_labels),
                  epochs=50, batch_size=64, shuffle=True,
                  callbacks=[early_stopping, model_checkpoint, tensorboard])

        return bst_model_path


    def preprocess_strings(self, some_queries):

        id_text_tr = []
        for text in some_queries:
            try:
                token_list = self.tokenizer.tokenize("[CLS] " + text + " [SEP]")
                segments_ids, indexed_tokens = [1] * len(token_list), self.tokenizer.convert_tokens_to_ids(token_list)
                id_text_tr.append(indexed_tokens)
            except KeyError:
                continue
        return id_text_tr


    def create_train_dev_set(self, sentences_pairs, is_similar, max_sequence_length, validation_split_ratio):

        sentences1 = [x[0].lower() for x in sentences_pairs]
        sentences2 = [x[1].lower() for x in sentences_pairs]

        train_sequences_1 = self.preprocess_strings(sentences1)
        train_sequences_2 = self.preprocess_strings(sentences2)

        leaks = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
                 for x1, x2 in zip(train_sequences_1, train_sequences_2)]

        train_padded_data_1 = pad_sequences(train_sequences_1, maxlen=max_sequence_length)
        train_padded_data_2 = pad_sequences(train_sequences_2, maxlen=max_sequence_length)

        train_labels = np.array(is_similar)
        leaks = np.array(leaks)

        shuffle_indices = np.random.permutation(np.arange(len(train_labels)))
        train_data_1_shuffled = train_padded_data_1[shuffle_indices]
        train_data_2_shuffled = train_padded_data_2[shuffle_indices]
        train_labels_shuffled = train_labels[shuffle_indices]
        leaks_shuffled = leaks[shuffle_indices]

        dev_idx = max(1, int(len(train_labels_shuffled) * validation_split_ratio))

        del train_padded_data_1
        del train_padded_data_2

        train_data_1, val_data_1 = train_data_1_shuffled[:-dev_idx], train_data_1_shuffled[-dev_idx:]
        train_data_2, val_data_2 = train_data_2_shuffled[:-dev_idx], train_data_2_shuffled[-dev_idx:]
        labels_train, labels_val = train_labels_shuffled[:-dev_idx], train_labels_shuffled[-dev_idx:]
        leaks_train, leaks_val = leaks_shuffled[:-dev_idx], leaks_shuffled[-dev_idx:]

        return train_data_1, train_data_2, labels_train, leaks_train, \
               val_data_1, val_data_2, labels_val, leaks_val


    def create_test_data(self, test_sentence_pairs, max_len):
        test_sentences1 = [x[0].lower() for x in test_sentence_pairs]
        test_sentences2 = [x[1].lower() for x in test_sentence_pairs]

        test_sequences_1 = self.preprocess_strings(test_sentences1)
        test_sequences_2 = self.preprocess_strings(test_sentences2)
        leaks_test = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
                      for x1, x2 in zip(test_sequences_1, test_sequences_2)]

        leaks_test = np.array(leaks_test)
        test_data_1 = pad_sequences(test_sequences_1, maxlen=max_len)
        test_data_2 = pad_sequences(test_sequences_2, maxlen=max_len)

        return test_data_1, test_data_2, leaks_test


class Solver21():

    def __init__(self, seed=42, path_to_model="data/models/siameise_model.h5"):
        self.has_model = True
        self.siamese = SiameseBiLSTM()
        self.best_model_path = path_to_model
        self.seed = seed
        self.init_seed()

    def init_seed(self):
        return random.seed(self.seed)

    def parse_task(self, task):
        """ link multiple_choice """
        assert task["question"]["type"] == "multiple_choice"

        choices = task["question"]["choices"]

        links, label = [], ""
        description = task["text"]

        if "двоеточие" in description:
            label = "двоеточие"
        if "тире" in description:
            label = "тире"
        if "запят" in description:
            label = "запятая"

        m = re.findall("[0-9]\\)", description)
        for n, match in enumerate(m, 1):
            first, description = description.split(match)
            if len(first) > 1 and "Найдите" not in first:
                links.append(first)
                if n == len(m):
                    description = description.split('\n')[0]
                    links.append(description.replace(' (', ''))

        assert len(links) == len(choices)

        return links, label

    def dash_task(self, choices):
        hypothesys = []

        for choice in choices:
            if ' –' in choice:
                hypothesys.append(choice)
            if " —" in choice:
                hypothesys.append(choice)
        return hypothesys

    def semicolon_task(self, choices):
        hypothesys = []
        for choice in choices:
            if ':' in choice:
                hypothesys.append(choice)
        return hypothesys

    def comma_task(self, choices):
        hypothesys = []
        for choice in choices:
            if ', ' in choice:
                hypothesys.append(choice)
        return hypothesys

    def fit(self, tasks):
        self.load(path="data/models/siameise_model.h5")
        return

        sentences_pairs, is_similar_target = [], []

        for task in tasks:
            choices, label = self.parse_task(task)
            y_true = task['solution']['correct_variants'] if 'correct_variants' in task['solution'] else [
                task['solution']['correct']]

            pairs, indexes = [], []
            for y in y_true[0]:
                for n, choice in enumerate(choices, 1):
                    if int(y) == n:
                        pairs.append(choice)
                    else:
                        indexes.append(choice)

            good_pairs = list(combinations(pairs, 2))

            for pair in good_pairs:
                sentences_pairs.append(pair)
                is_similar_target.append(1)

            bad_pair = indexes[:2]
            sentences_pairs.append(bad_pair)
            is_similar_target.append(0)

        self.best_model_path = self.siamese.train_model(
            sentences_pairs, is_similar_target
        )
        return self.best_model_path

    def predict_random(self, task):
        choices = task["question"]["choices"]

        pred = []
        for _ in range(random.choice([2, 3])):
            choice = random.choice(choices)
            pred.append(choice["id"])
            choices.remove(choice)
        return pred

    def predict(self, task):
        if not self.has_model:
            return self.predict_random(task)
        else:
            return self.predict_from_model(task)

    def load(self, path="data/models/siameise_model.h5"):
        print("Hi!, It's load")
        self.best_model_path = "data/models/siameise_model.h5"
        self.siamese_model_loaded = self.get_model()
        print("Siamese model is loaded")
        print(self.siamese.embedding_matrix)
        return self.siamese_model_loaded

    def save(self, path='data/models/siameise_model.h5'):
        pass

    @singleton
    def get_model(self):
        model = load_model(self.best_model_path)
        return model

    def predict_from_model(self, task):
        test_sentence_pairs = []
        choices, label = self.parse_task(task)
        choices_dict = {}
        for n, choice in enumerate(choices, 1):
            choices_dict[choice] = n

        if label == 'тире':
            hypothesys = self.dash_task(choices)
        elif label == 'запятая':
            hypothesys = self.comma_task(choices)
        else:
            hypothesys = self.semicolon_task(choices)

        for pair in list(combinations(hypothesys, 2)):
            test_sentence_pairs.append(pair)

        test_data_x1, test_data_x2, leaks_test = self.siamese.create_test_data(test_sentence_pairs, 40)

        try:
            preds = list(self.siamese_model_loaded.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
        except ValueError:
            preds = []

        preds = [pr for pr in preds]

        final_answer = []
        for pair, pred in zip(test_sentence_pairs, preds):

            max_pred = max(preds)
            if pred == max_pred:
                for n, choice in enumerate(choices, 1):
                    if choice == pair[0]:
                        final_answer.append(str(n))
                    if choice == pair[1]:
                        final_answer.append(str(n))
        if final_answer:
            return final_answer
        else:
            ch = [str(n) for n, choice in enumerate(choices, 1)]
            random.shuffle(ch)
            return ch[:2]


Using TensorFlow backend.


Решение тестов

In [0]:
import time
import warnings
import numpy as np
from utils import *
from solvers import *


def zero_if_exception(scorer):
    def new_scorer(*args, **kwargs):
        try:
            return scorer(*args, **kwargs)
        except:
            return 0
    return new_scorer  



class Evaluation(object):

    def __init__(self, train_path="public_set/train",
                 test_path="public_set/test",
                 score_path="data/evaluation/scoring.json"):
        self.train_path = train_path
        self.test_path = test_path
        self.score_path = score_path
        self.secondary_score = read_config(self.score_path)["secondary_score"]
        self.test_scores = []
        self.first_scores = []
        self.secondary_scores = []
        self.classifier = classifier.Solver()
        self.solvers = [
            solver1.Solver(),
            solver2.Solver(),
            solver3.Solver(),
            solver4.Solver(),
            Solver5(),
            solver6.Solver(),
            solver7.Solver(),
            Solver8(),
            solver9.Solver(),
            solver10.Solver(),
            solver10.Solver(),
            solver10.Solver(),
            solver13.Solver(),
            solver14.Solver(),
            solver15.Solver(),
            solver16.Solver(),
            solver17.Solver(train_size=0.9),
            solver17.Solver(train_size=0.85),
            solver17.Solver(train_size=0.85),
            solver17.Solver(train_size=0.85),
            solver21.Solver(),
            solver22.Solver(),
            solver23.Solver(),
            solver24.Solver(),
            solver25.Solver(),
            solver26.Solver()
        ]
        self.time_limit_is_ok = True
        time_limit_is_observed = self.solver_fitting()
        if time_limit_is_observed:
            print("Time limit of fitting is OK")
        else:
            self.time_limit_is_ok = False
            print("TIMEOUT: Some solvers fit longer than 10m!")
        self.clf_fitting()

    def solver_fitting(self):
        time_limit_is_observed = True
        for i, solver in enumerate(self.solvers):
            start = time.time()
            solver_index = i + 1
            train_tasks = load_tasks(self.train_path, task_num=solver_index)
            if hasattr(solver, "load"):
                print("Loading Solver {}".format(solver_index))
                solver.load("data/models/solver{}.pkl".format(solver_index))
            else:
                print("Fitting Solver {}...".format(solver_index))
                solver.fit(train_tasks)
            duration = time.time() - start
            if duration > 60:
                time_limit_is_observed = False
                print("Time limit is violated in solver {} which has been fitting for {}m {:2}s".format(
                    solver_index, int(duration // 60), duration % 60))
            print("Solver {} is ready!\n".format(solver_index))
        return time_limit_is_observed

    def clf_fitting(self):
        tasks = []
        for filename in os.listdir(self.train_path):
            if filename.endswith(".json"):
                data = read_config(os.path.join(self.train_path, filename))
                tasks.append(data)
        print("Fitting Classifier...")
        self.classifier.fit(tasks)
        print("Classifier is ready!")

    # для всех заданий с 1 баллом
    @zero_if_exception
    def get_score(self, y_true, prediction):
        if "correct" in y_true:
            if y_true["correct"] == prediction:
                return 1
        elif "correct_variants" in y_true and isinstance(y_true["correct_variants"][0], str):
            if  prediction in y_true["correct_variants"]:
                return 1
        elif "correct_variants" in y_true and isinstance(y_true["correct_variants"][0], list):
            y_true = set(y_true["correct_variants"][0])
            y_pred = set(prediction)
            return int(len(set.intersection(y_true, y_pred)) == len(y_true) == len(y_pred))
        return 0

    # для 8 и 26
    @zero_if_exception
    def get_matching_score(self, y_true, pred):
        score = 0
        y_true = y_true["correct"]
        if len(y_true) != len(pred):
            return 0
        for y in y_true:
            if y_true[y] == pred[y]:
                score += 1
        return score 

    # для 16 задания
    @zero_if_exception
    def get_multiple_score(self, y_true, y_pred):
        y_true = y_true["correct_variants"][0] if "correct_variants" in y_true else y_true["correct"]
        while len(y_pred) < len(y_true):
            y_pred.append(-1)
        return max(0, len(set.intersection(set(y_true), set(y_pred))) - len(y_pred) + len(y_true))

    def variant_score(self, variant_scores):
        first_score = sum(variant_scores)
        mean_score = round(np.mean(variant_scores), 3)
        secondary_score = int(self.secondary_score[str(first_score)])
        scores = {"first_score": first_score, "mean_accuracy": mean_score, "secondary_score": secondary_score}
        self.first_scores.append(first_score)
        self.secondary_scores.append(secondary_score)
        return scores

    def get_overall_scores(self):
        overall_scores = {}
        for variant, variant_scores in enumerate(self.test_scores):
            scores = self.variant_score(variant_scores)
            print("***YOUR RESULTS***")
            print("Variant: {}".format(variant + 1))
            print("Scores: {}\n".format(scores))
            overall_scores[str(variant + 1)] = scores
        self.overall_scores = overall_scores
        return self

    def predict_from_baseline(self):
        time_limit_is_observed = True
        for filename in os.listdir(self.test_path):
            predictions = []
            print("Solving {}".format(filename))
            data = read_config(os.path.join(self.test_path, filename))[:-1]
            task_number = self.classifier.predict(data)
            for i, task in enumerate(data):
                print(task)
                start = time.time()
                task_index, task_type = i + 1, task["question"]["type"]
                print("Predicting task {}...".format(task_index))
                y_true = task["solution"]
                prediction = self.solvers[task_number[i] - 1].predict_from_model(task)
                if task_type == "matching":
                    score = self.get_matching_score(y_true, prediction)
                elif task_index == 16:
                    score = self.get_multiple_score(y_true, prediction)
                else:
                    score = self.get_score(y_true, prediction)
                print("Score: {}\nCorrect: {}\nPrediction: {}\n".format(score, y_true, prediction))
                predictions.append(score)
                duration = time.time() - start
                if duration > 60:
                    time_limit_is_observed = False
                    self.time_limit_is_ok = False
                    print("Time limit is violated in solver {} which has been predicting for {}m {:2}s".format(
                        i+1, int(duration // 60), duration % 60))
            self.test_scores.append(predictions)
        return time_limit_is_observed


def main():
    warnings.filterwarnings("ignore")
    evaluation = Evaluation()
    time_limit_is_observed = evaluation.predict_from_baseline()
    if not time_limit_is_observed:
        print('TIMEOUT: some solvers predict longer then 60s!')
    evaluation.get_overall_scores()
    mean_first_score = np.mean(evaluation.first_scores)
    mean_secondary_score = np.mean(evaluation.secondary_scores)
    print("Mean First Score: {}".format(mean_first_score))
    print("Mean Secondary Score: {}".format(mean_secondary_score))

    if evaluation.time_limit_is_ok:
        print("Time limit is not broken by any of the solvers.")
    else:
        print("TIMEOUT: Time limit by violated in some of the solvers.")


In [112]:
main()

Loading Solver 1
Solver 1 is ready!

Loading Solver 2
Solver 2 is ready!

Loading Solver 3
Solver 3 is ready!

Loading Solver 4
Solver 4 is ready!

Loading Solver 5
Solver 5 is ready!

Loading Solver 6
Solver 6 is ready!

Loading Solver 7
Solver 7 is ready!

Loading Solver 8
Solver 8 is ready!

Loading Solver 9
Solver 9 is ready!

Loading Solver 10
Solver 10 is ready!

Loading Solver 11
Solver 11 is ready!

Loading Solver 12
Solver 12 is ready!

Loading Solver 13
Solver 13 is ready!

Loading Solver 14
Solver 14 is ready!

Loading Solver 15
Solver 15 is ready!

Loading Solver 16
Solver 16 is ready!

Loading Solver 17
Solver 17 is ready!

Loading Solver 18
Solver 18 is ready!

Loading Solver 19
Solver 19 is ready!

Loading Solver 20
Solver 20 is ready!

Loading Solver 21
Hi!, It's load
Siamese model is loaded
[[ 0.025951 -0.006173 -0.0041    0.038244 ...  0.015056  0.029652  0.024176  0.019703]
 [ 0.010381 -0.013629  0.006721  0.027971 ...  0.022037  0.012372  0.026722  0.033707]
 [ 0.02

Подготовка решателей

In [0]:
import random
from collections import defaultdict

from flask import Flask, request, jsonify
import numpy as np

from utils import *
from solvers import *

import traceback


solver_param = defaultdict(dict)
solver_param[17]["train_size"] = 0.9
solver_param[18]["train_size"] = 0.85
solver_param[19]["train_size"] = 0.85
solver_param[20]["train_size"] = 0.85


class CuttingEdgeStrongGeneralAI(object):

    def __init__(self, train_path='public_set/train'):
        self.train_path = train_path
        self.classifier = classifier.Solver()
        solver_classes = [
            solver1,
            solver2,
            solver3,
            solver4,
            solver5,
            solver6,
            solver7,
            solver8,
            solver9,
            solver10,
            solver10,
            solver10,
            solver13,
            solver14,
            solver15,
            solver16,
            solver17,
            solver17,
            solver17,
            solver17,
            solver21,
            solver22,
            solver23,
            solver24,
            solver25,
            solver26,
            solver27
        ]
        self.solvers = self.solver_loading(solver_classes)
        self.clf_fitting()

    def solver_loading(self, solver_classes):
        solvers = []
        for i, solver_class in enumerate(solver_classes):
            solver_index = i + 1
            train_tasks = load_tasks(self.train_path, task_num=solver_index)
            solver_path = os.path.join("data", "models", "solver{}.pkl".format(solver_index))
            solver = solver_class.Solver(**solver_param[solver_index])
            if os.path.exists(solver_path):
                print("Loading Solver {}".format(solver_index))
                solver.load(solver_path)
            else:
                print("Fitting Solver {}...".format(solver_index))
                try:
                    solver = solver_class.Solver(**solver_param[solver_index])
                    solver.fit(train_tasks)
                    solver.save(solver_path)
                except Exception as e:
                    print('Exception during fitting: {}'.format(e))
            print("Solver {} is ready!\n".format(solver_index))
            solvers.append(solver)
        return solvers

    def clf_fitting(self):
        tasks = []
        for filename in os.listdir(self.train_path):
            if filename.endswith(".json"):
                data = read_config(os.path.join(self.train_path, filename))
                tasks.append(data)
        print("Fitting Classifier...")
        self.classifier.fit(tasks)
        print("Classifier is ready!")
        return self

    def not_so_strong_task_solver(self, task):
        question = task['question']
        if question['type'] == 'choice':
            # pick a random answer
            choice = random.choice(question['choices'])
            answer = choice['id']
        elif question['type'] == 'multiple_choice':
            # pick a random number of random choices
            min_choices = question.get('min_choices', 1)
            max_choices = question.get('max_choices', len(question['choices']))
            n_choices = random.randint(min_choices, max_choices)
            random.shuffle(question['choices'])
            answer = [
                choice['id']
                for choice in question['choices'][:n_choices]
            ]
        elif question['type'] == 'matching':
            # match choices at random
            random.shuffle(question['choices'])
            answer = {
                left['id']: choice['id']
                for left, choice in zip(question['left'], question['choices'])
            }
        elif question['type'] == 'text':
            if question.get('restriction') == 'word':
                # pick a random word from the text
                words = [word for word in task['text'].split() if len(word) > 1]
                answer = random.choice(words)

            else:
                # random text generated with https://fish-text.ru
                answer = (
                    'Для современного мира реализация намеченных плановых заданий позволяет '
                    'выполнить важные задания по разработке новых принципов формирования '
                    'материально-технической и кадровой базы. Господа, реализация намеченных '
                    'плановых заданий играет определяющее значение для модели развития. '
                    'Сложно сказать, почему сделанные на базе интернет-аналитики выводы призывают '
                    'нас к новым свершениям, которые, в свою очередь, должны быть в равной степени '
                    'предоставлены сами себе. Ясность нашей позиции очевидна: базовый вектор '
                    'развития однозначно фиксирует необходимость существующих финансовых и '
                    'административных условий.'
                )

        else:
            raise RuntimeError('Unknown question type: {}'.format(question['type']))

        return answer

    def take_exam(self, exam):
        answers = {}
        # pprint.pprint(exam)
        if "tasks" in exam:
            variant = exam["tasks"]
            if isinstance(variant, dict):
                if "tasks" in variant.keys():
                    variant = variant["tasks"]
        else:
            variant = exam
        task_number = self.classifier.predict(variant)
        print("Classifier results: ", task_number)
        for i, task in enumerate(variant):
            task_id = task['id']
            task_index, task_type = i + 1, task["question"]["type"]
            try:
                prediction = self.solvers[task_number[i] - 1].predict_from_model(task)
                print("Prediction: ", prediction)
            except Exception as e:
                print(traceback.format_exc())
                prediction = self.not_so_strong_task_solver(task)
            if isinstance(prediction, np.ndarray):
                prediction = list(prediction)
            answers[task_id] = prediction
        return answers


app = Flask(__name__)

ai = CuttingEdgeStrongGeneralAI()


@app.route('/ready')
def http_ready():
    return 'OK'


@app.route('/take_exam', methods=['POST'])
def http_take_exam():
    request_data = request.get_json()
    answers = ai.take_exam(request_data)
    return jsonify({
        'answers': answers
    })


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)

Можно посмотреть на сам классификатор заданий

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.toktok import ToktokTokenizer
import random
import numpy as np
from sklearn.svm import LinearSVC
import utils
import os
from utils import read_config, load_pickle, save_pickle


class Solver(object):
    """
    Классификатор между заданиями.
    Работает на Tfidf векторах и мультиклассовом SVM.
    
    Parameters
    ----------
    seed : int, optional (default=42)
        Random seed.
    ngram_range : tuple, optional uple (min_n, max_n) (default=(1, 3))
        Used forTfidfVectorizer. 
        The lower and upper boundary of the range of n-values for different n-grams to be extracted.
        All values of n such that min_n <= n <= max_n will be used.
    num_tasks : int, optional (default=27)
        Count of all tasks.
        
   """

    def __init__(self, seed=42, ngram_range=(1, 3)):
        self.seed = seed
        self.ngram_range = ngram_range
        self.vectorizer = TfidfVectorizer(ngram_range=ngram_range)
        self.clf = LinearSVC(multi_class='ovr')
        self.init_seed()
        self.word_tokenizer = ToktokTokenizer()

    def init_seed(self):
        np.random.seed(self.seed)
        random.seed(self.seed)

    def predict(self, task):
        return self.predict_from_model(task)

    def fit(self, tasks):
        texts = []
        classes = []
        for data in tasks:
            for task in data:
                idx = int(task["id"])
                text = "{} {}".format(" ".join(self.word_tokenizer.tokenize(task['text'])), task['question']['type'])
                texts.append(text)
                classes.append(idx)
        vectors = self.vectorizer.fit_transform(texts)
        classes = np.array(classes)
        self.classes = np.unique(classes)
        self.clf.fit(vectors, classes)
        return self

    def predict_from_model(self, task):
        texts = []
        for task_ in task:
            text = "{} {}".format(" ".join(self.word_tokenizer.tokenize(task_['text'])), task_['question']['type'])
            texts.append(text)
        return self.clf.predict(self.vectorizer.transform(texts))
    
    def fit_from_dir(self, dir_path):
        tasks = []
        for file_name in os.listdir(dir_path):
            if file_name.endswith(".json"):
                data = read_config(os.path.join(dir_path, file_name))
                tasks.append(data)
        return self.fit(tasks)
    
    @classmethod
    def load(cls, path):
        return load_pickle(path)
    
    def save(self, path):
        save_pickle(self, path)

In [71]:
# Basic usage
from solvers import classifier
import json
from utils import read_config
clf = classifier.Solver()
tasks = []
dir_path = "public_set/train/"
for file_name in os.listdir(dir_path):
  if file_name.endswith(".json"):
    data = read_config(os.path.join(dir_path, file_name))
    tasks.append(data)
clf = clf.fit(tasks)
# Predict for last file in dir
numbers_of_tasks = clf.predict(read_config(os.path.join(dir_path, file_name)))
numbers_of_tasks

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [72]:
# Save classifier
clf.save("clf.pickle")
# Load classifier
clf.load("clf.pickle")

In [0]:
def load_task(filename, task_num=None):
    tasks=[]
    with open(filename, encoding='utf-8') as f:
      dt = f.read().encode('utf-8')
      data = json.loads(dt)
      tasks += [d for d in data if 'id' in d and int(d['id']) == task_num]
    return tasks

In [107]:
load_task("public_set/train/task_103.json", 21)

[{'attachments': [],
  'id': '21',
  'question': {'choices': [{'id': '1', 'link': '(1)'},
    {'id': '2', 'link': '(2)'},
    {'id': '3', 'link': '(3)'},
    {'id': '4', 'link': '(4)'},
    {'id': '5', 'link': '(5)'},
    {'id': '6', 'link': '(6)'},
    {'id': '7', 'link': '(7)'}],
   'min_choices': 1,
   'type': 'multiple_choice'},
  'score': 1,
  'solution': {'correct_variants': [['3', '6'], ['6', '3']]},
  'text': 'Найдите предложения, в которых тире ставится в соответствии с одним и тем же правилом пунктуации.\n(1) Ни лошади, ни дороги – ничего не видно. (2) «Завтра я уезжаю», – сказал он. (3) Любить – значит доверять друг другу. (4) Всё здесь: скамейки, трава, дорожки, уходящие в глубь парка, – было покрыто жёлтыми листьями. (5) С одной стороны, он прав, с другой – его правда принесла матери только огорчения. (6) Лишнее говорить – значит себе вредить. (7) Вот она наконец – безграничная, необозримая степь!\nЗапишите номера этих предложений.'}]